# Comparing sample results with those from _Maheu, et al._

In [3]:
using PyPlot, Printf, DelimitedFiles

include("../regression.jl")
include("../inference.jl")

include("../UpdateRule.jl")
include("../Leaky.jl")
include("../VarSMiLe.jl")
include("../ParticleFiltering.jl");

In [4]:
Random.seed!(1234);

## Item frequency

In [5]:
# generate sequence from given item frequency
function generate_seq_base(chunks, ps)
    seq = Array{Int32}(undef, sum(chunks))
    prob = Array{Float32}(undef, sum(chunks))
    
    i = 1
    for (p, chunk) in zip(ps, chunks)
        for j = 1:chunk
            seq[i] = rand() < p
            prob[i] = p
            i += 1
        end
    end
    
    return (seq, prob)
end;

In [6]:
function export_seq(seq, filename)
    f = open(filename, "w")

    for el in seq
        print(f, el)
        print(f, " ")
    end
    
    close(f)
end;

In [7]:
function read_seq(filename, len)
    f = open(filename, "r")
    t = readdlm(f)
    close(f)
    
    return t[1,1:len]
end;

In [8]:
function read_thetas_matlab(filename, len)
    f = open(filename, "r")
    t = readdlm(f)
    close(f)
    return t[1,1:len]
end;

In [9]:
# ============== DECODE BASE RATE ======================
# generate sequence
chunks = [200, 200, 200, 200, 200]
len = sum(chunks)

p = 0.25
ps = [p, 1-p, p, 1-p, p];

If the sequence already exists:

In [10]:
seq_base = read_seq("../../test/seq1.txt", 1000);
seq_base = Int.(seq_base);

If the sequence does not already exist:

In [11]:
(seq_base, prob) = generate_seq_base(chunks, ps)
export_seq(seq_base, "../../test/seq1.txt")
export_seq(prob, "../../test/seq1_p.txt")

In [11]:
sbf, thetas = decodesbftheta(seq_base, Leaky(Inf, prior_fixed(0)))

# thetas is the probability of each sequence element --> need to invert it so we always have P(x = 0)
thetas = vcat(abs.(thetas[1:end-1] .- seq_base)[2:end], abs(thetas[end] - seq_base[end]))
thetas_matlab = read_thetas_matlab("../../test/seq1_perfect_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [12]:
sbf, thetas = decodesbftheta(seq_base, Leaky(10, prior_fixed(0)))

# thetas is the probability of each sequence element --> need to invert it so we always have P(x = 0)
thetas = vcat(abs.(thetas[1:end-1] .- seq_base)[2:end], abs(thetas[end] - seq_base[end]))
thetas_matlab = read_thetas_matlab("../../test/seq1_leaky_10_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


## Transition probabilities

In [13]:
# generate sequence from given transition probabilities
function generate_seq_transition(chunks, ps1g2, ps2g1)
    seq = Array{Int32}(undef, sum(chunks))
    prob1g2 = Array{Float32}(undef, sum(chunks))
    prob2g1 = Array{Float32}(undef, sum(chunks))
    
    i = 2
    
    # set inital value
    seq[1] = rand() > 0.5
    prob1g2[1] = ps1g2[1]
    prob2g1[1] = ps2g1[1]
    
    for (p1g2, p2g1, chunk) in zip(ps1g2, ps2g1, chunks)
        for j = 1:chunk
            p = seq[i - 1] == 0 ? p2g1 : 1 - p1g2
            seq[i] = rand() < p
            prob1g2[i] = p1g2
            prob2g1[i] = p2g1
            i += 1
            if i > sum(chunks)
               break 
            end
        end
    end
    
    return (seq, prob1g2, prob2g1)
end;

In [14]:
# ============== DECODE TRANSITION PROBABILITIES ======================
# generate sequence
chunks = [100, 200, 100, 200]
len = sum(chunks)

p = 0.25
p1g2  = 1 .- [p, 1-p, p, 1-p]
p2g1  =      [p, 1-p, p, 1-p];

If the sequence already exists:

In [15]:
seq_trans = read_seq("../../test/seq2.txt", 600);
seq_trans = Int.(seq_trans);

If the sequence does not already exist:

In [ ]:
(seq_trans, prob1g2, prob2g1) = generate_seq_transition(chunks, p1g2, p2g1)
export_seq(seq_trans, "../../test/seq2.txt")
export_seq(prob1g2, "../../test/seq2_p1g2.txt")
export_seq(prob2g1, "../../test/seq2_p2g1.txt")

In [16]:
sbf, thetas = decodesbftheta(seq_trans, Leaky(Inf, prior_fixed(1)))

# thetas is the probability of each sequence element --> need to invert it so we always have P(x = 0)
thetas = vcat(abs.(thetas[1:end-1] .- seq_trans)[2:end], abs(thetas[end] - seq_trans[end]))
thetas_matlab = read_thetas_matlab("../../test/seq2_perfect_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [17]:
sbf, thetas = decodesbftheta(seq_trans, Leaky(16, prior_fixed(1), updateallcols=true))

# thetas is the probability of each sequence element --> need to invert it so we always have P(x = 0)
thetas = vcat(abs.(thetas[1:end-1] .- seq_trans)[2:end], abs(thetas[end] - seq_trans[end]))
thetas_matlab = read_thetas_matlab("../../test/seq2_leaky_16_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In the case m = 0 (item frequency learning), Maheu et al. estimate the thetas after seeing the value.

In the case m = 1 (transition frequency learning), Maheu et al. estimate the thetas before seeing the value, but discard the first value since there is no window before it.